In [5]:
import pandas as pd
from dotenv.parser import Position

from slt_positional_bias.dataset import generate_merged_data_frame, sort_data_frame, store_df_as_parquet, load_parquet_as_df, normalize_and_tokenize, jaccard, spearman_word_order_correlation

from slt_positional_bias.plots import savetable
from slt_positional_bias.features import sacrebleu_corpus, rouge_corpus, meteor_corpus, bertscore_verbose

df_10_name = "LLM-1 - Llama3 405 the best general model and big context size-sample-count-10-1-2025-08-10 00h-18m-01s"
df_20_name = "LLM-1 - Llama3 405 the best general model and big context size-sample-count-20-1-2025-08-10 14h-40m-19s"
df_30_name = "LLM-1 - Llama3 405 the best general model and big context size-sample-count-30-1-2025-08-10 16h-54m-56s"
df_40_name = "LLM-1 - Llama3 405 the best general model and big context size-sample-count-40-1-2025-08-10 18h-46m-08s"

df_10 = load_parquet_as_df(df_10_name)
df_20 = load_parquet_as_df(df_20_name)
df_30 = load_parquet_as_df(df_30_name)
df_40 = load_parquet_as_df(df_40_name)

df_10

,topic_id,topic,rel_3_doc_position,nr_rel_3_doc,nr_rel_0_doc,oracle,answer
0,2024-105741,"is it dangerous to have wbc over 15,000 withou...",0,1,9,Leukocytosis (Inpatient Care) – What You Need ...,"Having a WBC count over 15,000 without treatme..."
1,2024-105741,"is it dangerous to have wbc over 15,000 withou...",2,1,9,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
2,2024-105741,"is it dangerous to have wbc over 15,000 withou...",4,1,9,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
3,2024-105741,"is it dangerous to have wbc over 15,000 withou...",7,1,9,Leukocytosis (Inpatient Care) – What You Need ...,"Having a WBC count over 15,000 without treatme..."
4,2024-105741,"is it dangerous to have wbc over 15,000 withou...",9,1,9,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
...,...,...,...,...,...,...,...
275,2024-96063,how using maps can impact your pedagogy,0,1,9,Maps and map learning in social studies involv...,Using maps can have a significant impact on pe...
276,2024-96063,how using maps can impact your pedagogy,2,1,9,Maps and map learning in social studies involv...,Using maps can significantly impact pedagogy b...
277,2024-96063,how using maps can impact your pedagogy,4,1,9,Maps and map learning in social studies involv...,Using maps can significantly impact pedagogy b...
278,2024-96063,how using maps can impact your pedagogy,7,1,9,Maps and map learning in social studies involv...,Using maps can significantly impact pedagogy b...


In [6]:
for nd in [df_10, df_20, df_30, df_40]:
    nd.rename(columns={'rel_3_doc_position': 'Position of Oracle', 'nr_rel_0_doc': 'Number of Documents', 'oracle': 'references', 'answer': 'predictions'}, inplace=True)
    nd.drop(columns=['nr_rel_3_doc'], inplace=True)
    nd['Position of Oracle'] = nd['Position of Oracle'] + 1
    nd['Number of Documents'] = nd['Number of Documents'] + 1
    
df_40

,topic_id,topic,Position of Oracle,Number of Documents,references,predictions
0,2024-105741,"is it dangerous to have wbc over 15,000 withou...",1,40,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
1,2024-105741,"is it dangerous to have wbc over 15,000 withou...",10,40,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
2,2024-105741,"is it dangerous to have wbc over 15,000 withou...",20,40,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
3,2024-105741,"is it dangerous to have wbc over 15,000 withou...",30,40,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
4,2024-105741,"is it dangerous to have wbc over 15,000 withou...",40,40,Leukocytosis (Inpatient Care) – What You Need ...,"Having a white blood cell (WBC) count over 15,..."
...,...,...,...,...,...,...
275,2024-96063,how using maps can impact your pedagogy,1,40,Maps and map learning in social studies involv...,Using maps can significantly impact pedagogy b...
276,2024-96063,how using maps can impact your pedagogy,10,40,Maps and map learning in social studies involv...,Using maps can significantly impact pedagogy b...
277,2024-96063,how using maps can impact your pedagogy,20,40,Maps and map learning in social studies involv...,Using maps in pedagogy can have various impact...
278,2024-96063,how using maps can impact your pedagogy,30,40,Maps and map learning in social studies involv...,Using maps can significantly impact pedagogy b...
